In [1]:
using Revise

using Test
import TensorCrossInterpolation as TCI
import Random

In [2]:
using Test
import TensorCrossInterpolation as TCI
import TensorCrossInterpolation: rank, linkdims, TensorCI2, MultiIndex, evaluate, crossinterpolate2, pivoterror, tensortrain
import Random
import QuanticsGrids as QD

In [3]:
import TensorCrossInterpolation as TCI
using TensorCrossInterpolation
using Random
using Test
using ITensors
ITensors.disable_warn_order()

14

In [4]:
pivotsearch = :full
#seed = 12456
#seed = 1245678
seed = 124567890

Random.seed!(seed)

R = 10
abstol = 1e-2
sites = [Index(2, "n=$n") for n in 1:R]

Ψ = random_mps(sites; linkdims=20)
rindex = Set([rand(1:2, R) for _ in 1:20])

tensor = Array(reduce(*, Ψ), reverse(sites))
tensor = tensor ./ maximum(abs, tensor)

function f(x)
    res = tensor[x...]
    if x in rindex
        res += 1.01 * abstol
    end
    res
end

localdims = fill(2, R)
firstpivot = ones(Int, R)
tci, ranks, errors = crossinterpolate2(
    Float64,
    f,
    localdims,
    [firstpivot];
    tolerance=abstol,
    maxbonddim=1000,
    maxiter=20,
    loginterval=1,
    verbosity=1,
    normalizeerror=false,
    pivotsearch=pivotsearch,
)
@show abstol

(bondindex, length(Iset)) = (1, 2)
(bondindex, length(Iset)) = (2, 1)
(bondindex, length(Iset)) = (3, 2)
(bondindex, length(Iset)) = (4, 2)
(bondindex, length(Iset)) = (5, 2)
(bondindex, length(Iset)) = (6, 2)
(bondindex, length(Iset)) = (7, 2)
(bondindex, length(Iset)) = (8, 2)
(bondindex, length(Iset)) = (9, 2)
iteration = 1, rank = 12, error= 0.0007598560883149983, maxsamplevalue= 0.34207792474744125, nglobalpivot=0
(bondindex, length(Iset)) = (1, 2)
(bondindex, length(Iset)) = (2, 4)
(bondindex, length(Iset)) = (3, 6)
(bondindex, length(Iset)) = (4, 8)
(bondindex, length(Iset)) = (5, 10)
(bondindex, length(Iset)) = (6, 10)
(bondindex, length(Iset)) = (7, 6)
(bondindex, length(Iset)) = (8, 4)
(bondindex, length(Iset)) = (9, 2)
iteration = 2, rank = 37, error= 0.0, maxsamplevalue= 0.7670697949072186, nglobalpivot=0
(bondindex, length(Iset)) = (1, 2)
(bondindex, length(Iset)) = (2, 4)
(bondindex, length(Iset)) = (3, 8)
(bondindex, length(Iset)) = (4, 14)
(bondindex, length(Iset)) = (5

0.01

In [74]:
TCI.addglobalpivots2sitesweep!(
    tci, f, collect(rindex),
    tolerance=abstol,
    normalizeerror=false,
    maxbonddim=typemax(Int),
    pivotsearch=pivotsearch,
    verbosity=1,
)


(bondindex, length(Iset)) = (1, 2)
(bondindex, length(Iset)) = (2, 4)
(bondindex, length(Iset)) = (3, 8)
(bondindex, length(Iset)) = (4, 16)
(bondindex, length(Iset)) = (5, 26)
(bondindex, length(Iset)) = (6, 16)
(bondindex, length(Iset)) = (7, 8)
(bondindex, length(Iset)) = (8, 4)
(bondindex, length(Iset)) = (9, 2)
(bondindex, length(Iset)) = (1, 2)
(bondindex, length(Iset)) = (2, 4)
(bondindex, length(Iset)) = (3, 8)
(bondindex, length(Iset)) = (4, 16)
(bondindex, length(Iset)) = (5, 26)
(bondindex, length(Iset)) = (6, 16)
(bondindex, length(Iset)) = (7, 8)
(bondindex, length(Iset)) = (8, 4)
(bondindex, length(Iset)) = (9, 2)


In [75]:
tt = TCI.TensorTrain(TCI.sitetensors(tci, f))
@test sum(abs.([TCI.evaluate(tt, r) - f(r) for r in rindex]) .> abstol) == 0

orthocenter = 5


Test Passed

In [76]:
for b in 1:length(tci)-1
    @show length(TCI.Iset(tci, b+1)), length(TCI.Jset(tci, b))
end

(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (2, 61)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (4, 59)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (8, 53)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (16, 44)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (27, 30)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (38, 16)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (53, 8)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (59, 4)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (62, 2)


In [77]:
for orthocenter in [1, length(tci) ÷ 2]
#for orthocenter in [1]
    tt = TCI.TensorTrain(TCI.sitetensors(tci, f; orthocenter=orthocenter))
    TCI.compress!(tt, :SVD; tolerance=1e-10)
    println("Orthocenter: ", orthocenter)
    @show TCI.linkdims(tt)
    println("Error on pivots:")
    for b in 1:length(tci)-1
        Iset = TCI.Iset(tci, b+1)
        Jset = TCI.Jset(tci, b)
        diff = maximum(abs, [TCI.evaluate(tt, vcat(i, j)) - f(vcat(i, j)) for i in Iset, j in Jset])
        println(b, "    ", diff, " ", diff > abstol)
    end
    println("Error on T:")
    for l in 1:length(tci)-1
        diff = [TCI.evaluate(tt, vcat(i, m, j)) - f(vcat(i, m, j)) for i in TCI.Iset(tci, l), j in TCI.Jset(tci, l), m in 1:localdims[l]]
        println(l, "    ", maximum(abs, diff), " ", maximum(abs, diff) > abstol)
    end

    println("Error on Π:")
    for b in 1:length(tci)-1
        Iset_b = TCI.Iset(tci, b)
        Jset_bp1 = TCI.Jset(tci, b+1)
        diff = [
            TCI.evaluate(tt, vcat(i, i1, i2, j)) - f(vcat(i, i1, i2, j)) for i in Iset_b, j in Jset_bp1, i1 in 1:localdims[b], i2 in 1:localdims[b+1]]
        println(b, "    ", maximum(abs, diff), " ", maximum(abs, diff) > abstol)
    end

    println("error on global pivots: ", maximum([abs(TCI.evaluate(tt, p) - f(p)) for p in tci.globalpivots]))
end

orthocenter = 1
Orthocenter: 1
TCI.linkdims(tt) = [2, 4, 8, 16, 27, 16, 8, 4, 2]
Error on pivots:
1    0.0008948753827201783 false
2    0.0008948753827201783 false
3    0.0008948753827201783 false
4    0.0009328600859791897 false
5    4.385380947269368e-15 false
6    4.385380947269368e-15 false
7    4.385380947269368e-15 false
8    4.385380947269368e-15 false
9    4.218847493575595e-15 false
Error on T:
1    0.0008948753827201783 false
2    0.0008948753827201783 false
3    0.0008948753827201783 false
4    0.0009328600859791897 false
5    0.0016763381370814012 false
6    4.385380947269368e-15 false
7    4.385380947269368e-15 false
8    4.385380947269368e-15 false
9    4.385380947269368e-15 false
Error on Π:
1    0.0008948753827201783 false
2    0.0008948753827201783 false
3    0.0009328600859791897 false
4    0.0016763381370814012 false
5    0.0028238063684213888 false
6    4.385380947269368e-15 false
7    4.385380947269368e-15 false
8    4.385380947269368e-15 false
9    4.3853809472693

In [55]:
tt = TCI.TensorTrain(TCI.sitetensors(tci, f))
tt_old = TCI.TensorTrain(TCI.sitetensors_site0update(tci, f))

@show TCI.linkdims(tt)
@show TCI.linkdims(tt_old)

orthocenter = 5
TCI.linkdims(tt) = [2, 4, 8, 16, 28, 16, 8, 4, 2]
TCI.linkdims(tt_old) = [2, 4, 8, 16, 27, 16, 8, 4, 2]


9-element Vector{Int64}:
  2
  4
  8
 16
 27
 16
  8
  4
  2

In [56]:
for l in 1:length(tci)
    Iset = TCI.Iset(tci, l)
    Jset = TCI.Jset(tci, l)
    println(l, "    ", maximum(abs, [TCI.evaluate(tt, vcat(i, l, j)) - f(vcat(i, l, j)) for i in Iset, j in Jset, l in 1:localdims[l]]))
end

1    0.0013252755901388205
2    0.0013252755901388205
3    0.0013252755901388205
4    0.0014761629393269804
5    0.0015415748986997782
6    1.4432899320127035e-15
7    1.4432899320127035e-15
8    1.3322676295501878e-15
9    1.3322676295501878e-15
10    1.3322676295501878e-15


In [57]:
maximum([abs(TCI.evaluate(tt, p) - f(p)) for p in tci.globalpivots])

1.3322676295501878e-15